In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.master("local[4]") \
.appName("schema") \
.config("spark.executor.memory","4g") \
.config("spark.driver.memory","2g") \
.getOrCreate()
sc = spark.sparkContext

In [5]:
df = spark.read \
.option("header","True") \
.option("inferSchema","True")\
.option("sep",";") \
.csv("/home/taha/Downloads/OnlineRetail.csv")

In [6]:
df.show(2)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2,55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
only showing top 2 rows



In [7]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



# shema hazırlma 

In [8]:
from pyspark.sql.types import *

In [10]:
manuel_schema = StructType(
[
    StructField("InvoiceNo" , StringType() , True),
    StructField("StockCode" , StringType() , True),
    StructField("Description" , StringType() , True),
    StructField("Quantity" , IntegerType() , True),
    StructField("InvoiceDate" , StringType() , True),
    StructField("UnitPrice" , FloatType() , True),
    StructField("CustomerID" , IntegerType() , True),
    StructField("Country" , StringType() , True)
    
]
)

# Veriyi elle hazırlanan schema ile yeniden oku

In [11]:
df2 = spark.read \
.option("header","True") \
.schema(manuel_schema)\
.option("sep",";") \
.csv("/home/taha/Downloads/OnlineRetail.csv")

In [12]:
df2.show(2)

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|     null|     null|       null|    null|       null|     null|      null|   null|
|     null|     null|       null|    null|       null|     null|      null|   null|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
only showing top 2 rows



In [29]:
# null gelmesinin sebebi float , ile olamaz onu goremedi biz elle duzeltiyoruz
from pyspark.sql import functions as F
df2 = spark.read \
.option("header","True") \
.option("inferSchema","True")\
.option("sep",";") \
.csv("/home/taha/Downloads/OnlineRetail.csv") \
.withColumn("UnitPrice",F.regexp_replace(F.col("UnitPrice"),",","."))

In [30]:
df2.show(2)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
only showing top 2 rows



In [31]:
df2 \
.coalesce(1) \
.write \
.mode("overwrite") \
.option("sep",";") \
.option("header","true") \
.csv("/home/taha/Downloads/OnlineRetail2")

In [32]:
df2 = spark.read \
.option("header","True") \
.schema(manuel_schema)\
.option("sep",";") \
.csv("/home/taha/Downloads/OnlineRetail2") 

In [33]:
df2.show(2)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
only showing top 2 rows



In [34]:
df2.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: float (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)

